<a href="https://www.kaggle.com/code/ayushkhaire/crypto-dataset-daily-updates-top-1000?scriptVersionId=212816793" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import requests as rq
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
from kaggle_secrets import UserSecretsClient
from datetime import datetime
import json as js
import subprocess
import logging
import time
import shutil

# Yeasterday data :)

In [2]:
df = pd.read_csv('/kaggle/input/top-1000-cryptos-historical/1000_cryptos.csv')
df

,dates,symbol,open,close,high,low,volume,adj_close
0,2021-09-16 00:00:00,PBX-USD,0.000000,0.022560,0.029812,0.000000,60069068.0,0.022560
1,2021-09-17 00:00:00,PBX-USD,0.022413,0.016698,0.023667,0.014529,28618681.0,0.016698
2,2021-09-18 00:00:00,PBX-USD,0.016693,0.017480,0.018452,0.014744,12411074.0,0.017480
3,2021-09-19 00:00:00,PBX-USD,0.017483,0.019219,0.020695,0.015712,12801514.0,0.019219
4,2021-09-20 00:00:00,PBX-USD,0.019166,0.014756,0.019213,0.014603,7945956.0,0.014756
...,...,...,...,...,...,...,...,...
689209,2024-12-06 00:00:00,BCN-USD,0.000053,0.000051,0.000061,0.000045,4223.0,0.000051
689210,2024-12-07 00:00:00,BCN-USD,0.000051,0.000051,0.000058,0.000045,5436.0,0.000051
689211,2024-12-08 00:00:00,BCN-USD,0.000051,0.000051,0.000060,0.000051,4349.0,0.000051
689212,2024-12-09 00:00:00,BCN-USD,0.000051,0.000048,0.000053,0.000048,1957.0,0.000048


# Define headers

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Extract symbols

In [4]:
 link = f'https://finance.yahoo.com/markets/crypto/all/?start=0&count=100'
link

'https://finance.yahoo.com/markets/crypto/all/?start=0&count=100'

# Get symbols and hit urls

In [5]:
resp_main_list = []
i = 0
for page in tqdm(range(20)):
    url = f'https://finance.yahoo.com/markets/crypto/all/?start={i*100}&count=100'
    r = rq.get(url,headers = headers)
    resp_main_list.append(r)
    i = i+1
#     print(url)

100%|██████████| 20/20 [00:16<00:00,  1.24it/s]


In [6]:
symbol_list = []

for r in tqdm(resp_main_list):
    soup = BeautifulSoup(r.text,'html.parser')
    symbs= soup.find_all('span',{'class':'symbol'})
    for s in symbs:
        symbol_list.append(s.text)
        
print(len(symbol_list))

100%|██████████| 20/20 [00:08<00:00,  2.29it/s]

2520


# Now , get the link for today downloads from a tesla stock 

### This we willl use to use "code" by yahoo finance to figure out dates 

In [7]:
'''

llink1 = https://query1.finance.yahoo.com/v8/finance/chart/GLM-USD?events=capitalGain%7Cdiv%7Csplit&formatted=true&includeAdjustedClose=true&interval=1d&period1=1510185600&period2=1725897789&symbol=GLM-USD&userYfid=true&lang=en-US&region=US

here the period values are exactly same.
'''

# After research , i got they are unix timestamps in period . lets try it out !

import time

# Get current Unix timestamp
current_timestamp = int(time.time())

date_str = "2015-01-01"
date_obj = datetime.strptime(date_str, "%Y-%m-%d")
period1 = int(time.mktime(date_obj.timetuple()))

period2 = current_timestamp  

# Print the timestamps in the desired format
print(f"period1={period1}&period2={period2}")


period1=1420070400&period2=1734070020


# create stocks directory

In [8]:
os.mkdir('crypto_data')

# Download all files

In [9]:
def return_timestamp(unix_timestamps, date_format='%Y-%m-%d %H:%M:%S'):
    formatted_dates = []
    for ut in unix_timestamps:
        date = datetime.fromtimestamp(ut)
        formatted_date = date.strftime(date_format)
        formatted_dates.append(formatted_date)
    return formatted_dates

print(return_timestamp([1725898133]))

['2024-09-09 16:08:53']


In [10]:
for stock in tqdm(symbol_list):
    url = f'https://query1.finance.yahoo.com/v8/finance/chart/{stock}?events=capitalGain%7Cdiv%7Csplit&formatted=true&includeAdjustedClose=true&interval=1d&period1={period1}&period2={period2}&symbol={stock}&userYfid=true&lang=en-US&region=US'
#     print(url)
    r = rq.get(url, headers=headers)
    if r.status_code == 200:
        with open(f"crypto_data/{stock}.json", "wb") as file:
            file.write(r.content)
#         print(f"File downloaded successfully : {stock}.json")
    else:
#         print(f"Failed to download file : {stock}.json")
        pass
#     time.sleep(3)

100%|██████████| 2520/2520 [10:53<00:00,  3.86it/s]


In [11]:
symbol_list_dir = os.listdir('/kaggle/working/crypto_data/')
len(symbol_list_dir)

19

# Read a sample

In [12]:
# jsonf = pd.read_json(f'/kaggle/working/crypto_data/BTC-USD.json')
# jsondict = jsonf.to_dict()
# timestamp = jsondict.get('chart').get('result')[0].get('timestamp')
# sym = jsondict.get('chart').get('result')[0].get('meta').get('symbol')
# symbol = list(sym for s in range(len(timestamp)))
# low = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('low')
# high = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('high')
# volume = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('volume')
# open_ = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('open')
# close = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('close')
# adj_close = jsondict.get('chart').get('result')[0].get('indicators').get('adjclose')[0].get('adjclose')
# dates = return_timestamp(timestamp)
# tempdf = pd.DataFrame(
#     {
#     'dates':dates,
#     'symbol':symbol,
#     'open':open_,
#     'close':close,
#     'high':high,
#     'low':low,
#     'volume':volume,
#     'adj_close':adj_close
#     }
# )
# tempdf

# Work on dataset

In [13]:
megadf = pd.DataFrame()

In [14]:
for jso in tqdm(symbol_list_dir):
    jsonf = pd.read_json(f'/kaggle/working/crypto_data/{jso}')
    jsondict = jsonf.to_dict()
    
    # Safely get the timestamp and check if it's None
    timestamp = jsondict.get('chart').get('result')[0].get('timestamp')
    if timestamp is None:
        print(f"Missing timestamp for {jso}. Skipping this symbol.")
        continue  # Skip this iteration if timestamp is None
    
    # Proceed with the rest of the code after ensuring timestamp is not None
    low = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('low')
    sym = jsondict.get('chart').get('result')[0].get('meta').get('symbol')
    
    # Create a list of symbols based on the length of timestamp
    symbol = list(sym for s in range(len(timestamp)))
    
    high = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('high')
    volume = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('volume')
    open_ = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('open')
    close = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('close')
    adj_close = jsondict.get('chart').get('result')[0].get('indicators').get('adjclose')[0].get('adjclose')
    
    # Convert timestamp to dates
    dates = return_timestamp(timestamp)
    
    # Create a DataFrame
    tempdf = pd.DataFrame(
    {
        'dates': dates,
        'symbol': symbol,
        'open': open_,
        'close': close,
        'high': high,
        'low': low,
        'volume': volume,
        'adj_close': adj_close
    })
    
    # Concatenate the new data to the main DataFrame
    megadf = pd.concat([megadf, tempdf], ignore_index=True)


100%|██████████| 19/19 [00:00<00:00, 67.94it/s]


In [15]:
megadf['symbol'].unique()

array(['COST', 'AGIO', 'RIOT', 'BTDR', 'EQX', 'NTNX', 'NVDA', 'WVE',
       'ADBE', 'CELH', 'TSLA', 'UNH', 'UBER', 'GOLD', 'CIEN', 'CADL',
       'WBD', 'LCID', 'FTAI'], dtype=object)

In [16]:
first_date = megadf['dates'][0]
last_date = megadf['dates'].iloc[-1]

In [17]:
first_date,last_date

('2015-01-02 14:30:00', '2024-12-12 14:30:00')

In [18]:
megadf.dtypes

dates         object
symbol        object
open         float64
close        float64
high         float64
low          float64
volume         int64
adj_close    float64
dtype: object

In [19]:
megadf['dates'] = pd.to_datetime(megadf['dates'], format="mixed")

In [20]:
megadf.dtypes

dates        datetime64[ns]
symbol               object
open                float64
close               float64
high                float64
low                 float64
volume                int64
adj_close           float64
dtype: object

In [21]:
megadf

,dates,symbol,open,close,high,low,volume,adj_close
0,2015-01-02 14:30:00,COST,141.869995,141.610001,143.149994,140.809998,1471000,114.731621
1,2015-01-05 14:30:00,COST,141.690002,140.000000,142.160004,139.800003,1748500,113.427185
2,2015-01-06 14:30:00,COST,140.610001,141.850006,142.940002,139.850006,3185200,114.926010
3,2015-01-07 14:30:00,COST,142.600006,144.320007,144.429993,142.360001,2171600,116.927231
4,2015-01-08 14:30:00,COST,145.559998,145.559998,146.100006,144.940002,2937700,117.931831
...,...,...,...,...,...,...,...,...
39667,2024-12-06 14:30:00,FTAI,158.539993,157.110001,158.539993,154.104996,1699300,157.110001
39668,2024-12-09 14:30:00,FTAI,157.149994,149.899994,158.399994,149.000000,1579900,149.899994
39669,2024-12-10 14:30:00,FTAI,150.210007,150.000000,153.434998,149.369995,1351000,150.000000
39670,2024-12-11 14:30:00,FTAI,152.610001,149.669998,152.610001,149.350006,932800,149.669998


In [22]:
os.mkdir('dataset')

In [23]:
megadf.to_csv(f'dataset/1000_cryptos.csv',index = False)

# final dataset

In [24]:
df = pd.read_csv(f'/kaggle/working/dataset/1000_cryptos.csv')

In [25]:
df

,dates,symbol,open,close,high,low,volume,adj_close
0,2015-01-02 14:30:00,COST,141.869995,141.610001,143.149994,140.809998,1471000,114.731621
1,2015-01-05 14:30:00,COST,141.690002,140.000000,142.160004,139.800003,1748500,113.427185
2,2015-01-06 14:30:00,COST,140.610001,141.850006,142.940002,139.850006,3185200,114.926010
3,2015-01-07 14:30:00,COST,142.600006,144.320007,144.429993,142.360001,2171600,116.927231
4,2015-01-08 14:30:00,COST,145.559998,145.559998,146.100006,144.940002,2937700,117.931831
...,...,...,...,...,...,...,...,...
39667,2024-12-06 14:30:00,FTAI,158.539993,157.110001,158.539993,154.104996,1699300,157.110001
39668,2024-12-09 14:30:00,FTAI,157.149994,149.899994,158.399994,149.000000,1579900,149.899994
39669,2024-12-10 14:30:00,FTAI,150.210007,150.000000,153.434998,149.369995,1351000,150.000000
39670,2024-12-11 14:30:00,FTAI,152.610001,149.669998,152.610001,149.350006,932800,149.669998


# Upload Dataset

# User secrets

In [26]:
user_secrets = UserSecretsClient()
kaggle_apikey = user_secrets.get_secret("kaggle_apikey")
kaggle_username = user_secrets.get_secret("kaggle_username")

os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_apikey

# Metadata file

In [27]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print('Starting upload >>>>>>') 
print('Creating metadata file >>>>')
data = {
    "id": "ayushkhaire/top-1000-cryptos-historical"
}
metadata_file_location = '/kaggle/working/dataset/dataset-metadata.json' 
with open(metadata_file_location, 'w', encoding='utf-8') as metadata_file:
    js.dump(data, metadata_file)
print('Metadata file created')

Starting upload >>>>>>
Creating metadata file >>>>
Metadata file created


# upload

In [28]:
retries = 0
while retries < 5:
    try:
        command = "kaggle datasets version -p '/kaggle/working/dataset' -m 'Update' -r zip"
        subprocess.run(command, shell=True, check=True)
        logger.info("Upload complete")
        break
    except Exception as error:
        logger.error(f"Error from Kaggle: {error}")
        time.sleep(5)
        retries += 1

Starting upload for file 1000_cryptos.csv


100%|██████████| 4.58M/4.58M [00:02<00:00, 2.36MB/s]


Upload successful: 1000_cryptos.csv (5MB)
Dataset version is being created. Please check progress at https://www.kaggle.com/ayushkhaire/top-1000-cryptos-historical


In [29]:
shutil.rmtree('/kaggle/working/crypto_data')
shutil.rmtree('/kaggle/working/dataset')

# Thank you , and I will see you in next notebook :)